In [15]:
import pandas as pd
import random
from collections import defaultdict

# Complete color palette with psychological attributes
COLOR_PALETTE = {
    'shirt': [
        'Pink', 'Light Pink', 'Dark Pink', 'Red', 'Light Red', 'Dark Red',
        'Blue', 'Light Blue', 'Dark Blue', 'Green', 'Light Green', 'Dark Green',
        'Yellow', 'Light Yellow', 'Dark Yellow', 'Orange', 'Light Orange', 'Dark Orange',
        'Purple', 'Light Purple', 'Dark Purple', 'White', 'Gray', 'Light Gray', 'Dark Gray',
        'Black', 'Beige', 'Light Beige', 'Dark Beige'
    ],
    'pants': [
        'Black', 'Dark Gray', 'Gray', 'Light Gray', 'White', 'Dark Blue', 'Blue',
        'Dark Brown', 'Brown', 'Light Brown', 'Beige', 'Dark Beige', 'Khaki',
        'Dark Green', 'Olive', 'Charcoal'
    ],
    'shoes': [
        'Black', 'Dark Brown', 'Brown', 'Tan', 'White', 'Gray', 'Dark Gray',
        'Oxblood', 'Burgundy', 'Dark Blue', 'Dark Green'
    ],
    'jacket': [
        'Black', 'Navy', 'Dark Gray', 'Charcoal', 'Brown', 'Dark Brown', 'Beige',
        'Light Gray', 'White', 'Olive', 'Dark Green', 'Camel', 'Tan'
    ]
}

# Psychological attributes (Energy, Calmness, Professionalism)
COLOR_PSYCHOLOGY = {
    'Pink': (0.7, 0.7, 0.6),
    'Light Pink': (0.6, 0.8, 0.5),
    'Dark Pink': (0.7, 0.6, 0.7),
    'Red': (0.9, 0.3, 0.7),
    'Light Red': (0.8, 0.5, 0.6),
    'Dark Red': (0.7, 0.6, 0.8),
    'Blue': (0.6, 0.8, 0.9),
    'Light Blue': (0.5, 0.9, 0.8),
    'Dark Blue': (0.7, 0.7, 1.0),
    'Green': (0.5, 0.8, 0.7),
    'Light Green': (0.4, 0.9, 0.6),
    'Dark Green': (0.6, 0.7, 0.8),
    'Yellow': (0.9, 0.5, 0.5),
    'Light Yellow': (0.8, 0.7, 0.6),
    'Dark Yellow': (0.7, 0.6, 0.7),
    'Orange': (0.8, 0.5, 0.6),
    'Light Orange': (0.7, 0.7, 0.5),
    'Dark Orange': (0.7, 0.6, 0.7),
    'Purple': (0.7, 0.6, 0.8),
    'Light Purple': (0.6, 0.8, 0.7),
    'Dark Purple': (0.7, 0.7, 0.9),
    'White': (0.5, 0.9, 0.9),
    'Gray': (0.3, 0.8, 0.9),
    'Light Gray': (0.4, 0.9, 0.8),
    'Dark Gray': (0.3, 0.7, 1.0),
    'Black': (0.2, 0.7, 1.0),
    'Beige': (0.4, 0.9, 0.7),
    'Light Beige': (0.3, 1.0, 0.6),
    'Dark Beige': (0.4, 0.8, 0.8)
}

def get_psychology(color):
    base_color = color.split(' ')[-1]  # Handle 'Light/Dark' prefixes
    return COLOR_PSYCHOLOGY.get(base_color, (0.5, 0.5, 0.7))

def color_harmony(color1, color2):
    # Complementary colors
    complements = {
        'Red': 'Green',
        'Blue': 'Orange',
        'Yellow': 'Purple',
        'Green': 'Red',
        'Orange': 'Blue',
        'Purple': 'Yellow'
    }

    base1 = color1.split(' ')[-1]
    base2 = color2.split(' ')[-1]

    # Check complements
    if base1 in complements and complements[base1] == base2:
        return 2.5
    if base2 in complements and complements[base2] == base1:
        return 2.5

    # Analogous colors (same color family)
    color_families = {
        'Red': ['Pink', 'Red', 'Orange'],
        'Blue': ['Blue', 'Light Blue', 'Dark Blue', 'Navy'],
        'Green': ['Green', 'Light Green', 'Dark Green', 'Olive'],
        'Neutral': ['Black', 'White', 'Gray', 'Beige', 'Brown', 'Tan']
    }

    for family in color_families.values():
        if base1 in family and base2 in family:
            return 2.0 if abs(family.index(base1) - family.index(base2)) == 1 else 1.8

    # Light-Dark contrast
    lightness1 = 0 if 'Light' in color1 else (1 if 'Dark' in color1 else 0.5)
    lightness2 = 0 if 'Light' in color2 else (1 if 'Dark' in color2 else 0.5)
    if abs(lightness1 - lightness2) > 0.6:
        return 1.7

    return 1.0  # Neutral combination

def calculate_outfit_score(shirt, pants, shoes, jacket):
    # Color harmony (weighted)
    harmony = (
        3.0 * color_harmony(shirt, pants) +
        2.0 * color_harmony(pants, shoes) +
        2.0 * color_harmony(shirt, jacket) +
        1.5 * color_harmony(jacket, pants)
    )

    # Psychological balance
    shirt_e, shirt_c, shirt_p = get_psychology(shirt)
    pants_e, pants_c, pants_p = get_psychology(pants)
    shoes_e, shoes_c, shoes_p = get_psychology(shoes)
    jacket_e, jacket_c, jacket_p = get_psychology(jacket)

    # Calculate weighted scores
    energy = 0.4 * shirt_e + 0.2 * pants_e + 0.1 * shoes_e + 0.3 * jacket_e
    calm = 0.4 * shirt_c + 0.2 * pants_c + 0.1 * shoes_c + 0.3 * jacket_c
    professional = 0.4 * shirt_p + 0.2 * pants_p + 0.1 * shoes_p + 0.3 * jacket_p

    # Penalize too many same-hue colors
    hue_counts = defaultdict(int)
    for color in [shirt, pants, shoes, jacket]:
        base_color = color.split(' ')[-1]
        hue_counts[base_color] += 1
    hue_penalty = sum(max(0, count-1) * 0.3 for count in hue_counts.values())

    # Calculate final score
    mood_balance = 2.5 - abs(energy - 0.6) - abs(calm - 0.7)
    prof_score = professional * 1.5
    final_score = harmony * mood_balance * prof_score - hue_penalty

    # Ensure the final score and other metrics are properly returned
    return (round(max(0, final_score), 2), round(energy, 2), round(calm, 2), round(professional, 2))


def generate_outfits(num_outfits=4):
    used_colors = defaultdict(set)
    outfits = []

    # Generate candidates
    candidates = []
    for _ in range(3000):
        shirt = random.choice(COLOR_PALETTE['shirt'])
        pants = random.choice(COLOR_PALETTE['pants'])
        shoes = random.choice(COLOR_PALETTE['shoes'])
        jacket = random.choice(COLOR_PALETTE['jacket'])

        # Skip invalid combinations
        if pants == jacket or shoes == jacket:
            continue

        score, energy, calm, prof = calculate_outfit_score(shirt, pants, shoes, jacket)
        candidates.append((score, shirt, pants, shoes, jacket, energy, calm, prof))

    # Sort and select unique outfits
    candidates.sort(reverse=True, key=lambda x: x[0])

    for candidate in candidates:
        score, shirt, pants, shoes, jacket, energy, calm, prof = candidate

        # Check color uniqueness
        colors = {shirt, pants, shoes, jacket}
        if (not colors.intersection(used_colors['shirt']) and
            not colors.intersection(used_colors['pants']) and
            not colors.intersection(used_colors['shoes']) and
            not colors.intersection(used_colors['jacket'])):

            outfits.append({
                'shirt': shirt,
                'pants': pants,
                'shoes': shoes,
                'jacket': jacket,
                'score': score,
                'energy': energy,
                'calm': calm,
                'professional': prof
            })

            # Mark colors as used
            used_colors['shirt'].add(shirt)
            used_colors['pants'].add(pants)
            used_colors['shoes'].add(shoes)
            used_colors['jacket'].add(jacket)

            if len(outfits) >= num_outfits:
                break

    return pd.DataFrame(outfits)

# Generate and display outfits
outfits = generate_outfits()
print("Top 4 Psychologically-Optimized Outfit Combinations")
print(outfits[['shirt', 'pants', 'shoes', 'jacket', 'score', 'energy', 'calm', 'professional']].to_string(index=False))

Top 4 Psychologically-Optimized Outfit Combinations
      shirt     pants     shoes     jacket  score  energy  calm  professional
      White Dark Gray     White      Black  49.79    0.37  0.82          0.93
 Light Gray     Beige     Brown Light Gray  44.67    0.34  0.79          0.84
Light Beige      Gray      Gray Dark Brown  41.51    0.40  0.75          0.76
Dark Orange      Blue Dark Blue Dark Green  38.02    0.65  0.68          0.72
